In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [10]:
mnist_df, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_df['train'], mnist_df['test']

In [11]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image = image/255.
    return image, label

scaled_train_and_validation_df = mnist_train.map(scale)

scaled_test_df = mnist_test.map(scale)

In [12]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_df = scaled_train_and_validation_df.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_df.take(num_validation_samples)
train_data = shuffled_train_and_validation_df.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_df.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [18]:
input_size = 784
output_size = 10
hidden_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_size, activation ='relu'),
                            tf.keras.layers.Dense(hidden_size, activation ='relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax')
                            ])

In [19]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [20]:
num_epochs = 5

model.fit(train_data, epochs = num_epochs, validation_data = (validation_inputs, validation_targets))

Epoch 1/5
540/540 [==============================] - 5s 7ms/step - loss: 0.6175 - accuracy: 0.8247 - val_loss: 0.1609 - val_accuracy: 0.9572
Epoch 2/5
540/540 [==============================] - 4s 7ms/step - loss: 0.1432 - accuracy: 0.9577 - val_loss: 0.1072 - val_accuracy: 0.9670
Epoch 3/5
540/540 [==============================] - 4s 7ms/step - loss: 0.1024 - accuracy: 0.9697 - val_loss: 0.0835 - val_accuracy: 0.9737
Epoch 4/5
540/540 [==============================] - 5s 8ms/step - loss: 0.0781 - accuracy: 0.9751 - val_loss: 0.0771 - val_accuracy: 0.9760
Epoch 5/5
540/540 [==============================] - 4s 7ms/step - loss: 0.0632 - accuracy: 0.9803 - val_loss: 0.0542 - val_accuracy: 0.9825


In [22]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 910ms/step - loss: 0.0854 - accuracy: 0.9731
